<a href="https://colab.research.google.com/github/Vicysaragih/IDCamp-2024-Level-Menengah/blob/main/IDCamp_2024_Level_Menengah_11_Feb_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Studi Kasus Implementasi Klasifikasi Teks pada NLP: Sentimen Analisis Review APK Play Store**

# **Import Library**

In [1]:
!pip install google-play-scraper

# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews, Sort, reviews_all

import pandas as pd  # Pandas untuk manipulasi dan analisis data
pd.options.mode.chained_assignment = None  # Menonaktifkan peringatan chaining
import numpy as np  # NumPy untuk komputasi numerik
seed = 0
np.random.seed(seed)  # Mengatur seed untuk reproduktibilitas
import matplotlib.pyplot as plt  # Matplotlib untuk visualisasi data
import seaborn as sns  # Seaborn untuk visualisasi data statistik, mengatur gaya visualisasi
from sklearn.metrics import accuracy_score

import datetime as dt  # Manipulasi data waktu dan tanggal
import re  # Modul untuk bekerja dengan ekspresi reguler
import string  # Berisi konstanta string, seperti tanda baca
from nltk.tokenize import word_tokenize  # Tokenisasi teks
from nltk.corpus import stopwords  # Daftar kata-kata berhenti dalam teks

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Stemming (penghilangan imbuhan kata) dalam bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Menghapus kata-kata berhenti dalam bahasa Indonesia

from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 10.8 MB/s eta 0:00:00


# **Scraping Dataset**

Scraping data adalah cara untuk mengambil informasi dari halaman web dengan otomatis

In [2]:
# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews_all, Sort

# Mengambil semua ulasan dari aplikasi dengan ID 'com.byu.id' di Google Play Store.
# Proses scraping mungkin memerlukan beberapa saat tergantung pada jumlah ulasan yang ada.
scrapreview = reviews_all(
    'com.byu.id',          # ID aplikasi
    lang='id',             # Bahasa ulasan (default: 'en')
    country='id',          # Negara (default: 'us')
    sort=Sort.MOST_RELEVANT, # Urutan ulasan (default: Sort.MOST_RELEVANT)
    count=1000             # Jumlah maksimum ulasan yang ingin diambil
)

In [3]:
# Menyimpan ulasan dalam file CSV
import csv

with open('ulasan_aplikasi.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Review'])  # Menulis header kolom
    for review in scrapreview:
        writer.writerow([review['content']])  # Menulis konten ulasan ke dalam file CSV

Kode ini untuk menyimpan ulasan sebuah aplikasi dalam file CSV. Pertama, kita membuka file CSV baru untuk penulisan. Kemudian, kita menambahkan header kolom "Review". Selanjutnya, kita iterasi melalui setiap ulasan dalam variabel scrapreview dan menulis konten ulasan dalam file CSV. Jadi, file CSV akan berisi semua ulasan aplikasi dengan struktur yang sesuai.

# **Loading Dataset**

In [4]:
app_reviews_df = pd.DataFrame(scrapreview)
app_reviews_df.shape
app_reviews_df.head()
app_reviews_df.to_csv('ulasan_aplikasi.csv', index=False)

# Membuat DataFrame dari hasil scrapreview
app_reviews_df = pd.DataFrame(scrapreview)

# Menghitung jumlah baris dan kolom dalam DataFrame
jumlah_ulasan, jumlah_kolom = app_reviews_df.shape

Kode ini untuk mengambil ulasan aplikasi menggunakan scrapreview dan menyimpannya dalam sebuah DataFrame. Setelah itu, kita menampilkan jumlah baris dan kolom dalam dataset serta lima baris pertama untuk tinjauan cepat. Akhirnya, dataset disimpan dengan format CSV dan nama 'ulasan_aplikasi.csv' untuk digunakan dalam analisis selanjutnya.

In [5]:
# Menampilkan lima baris pertama dari DataFrame app_reviews_df
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,87509e9b-0076-4eb7-9c74-83ee5b8456ef,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"sangat buruk,setiap membeli kuota dengan apk i...",1,156,1.59.0,2025-01-16 14:07:40,"Hai Kak, maaf udah bikin ga nyaman ya :( Kalau...",2025-01-16 21:11:59,1.59.0
1,1b1ca5ac-66bd-4613-9bb4-37cb1c3318f2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Sering hilang sinyal,lemot dan parah sinyalnya...",1,10,1.60.0,2025-02-06 04:03:59,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2025-02-06 06:23:12,1.60.0
2,ff7da062-d90e-447c-afad-97a488dcf1ea,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Aplikasi berat banget, suka bengong. Kode prog...",1,47,1.60.0,2025-01-31 22:23:53,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2025-02-01 18:36:51,1.60.0
3,ef265867-a243-4a97-bc6a-4209d51b2525,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Kualitas Jaringan sangat lemot dan sering terg...,1,18,1.59.0,2025-01-26 07:37:31,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2025-01-26 15:22:54,1.59.0
4,02cfdf8a-a35e-449c-a92e-c5c9668dd08c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Bagi kalian yang baca ini terus jaringan byu l...,1,15,1.60.0,2025-02-02 20:34:20,"Hai Kak, maaf nih terkait kendalanya, bisa cob...",2025-02-02 23:00:18,1.60.0


In [6]:
# Menampilkan informasi tentang DataFrame app_reviews_df
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148500 entries, 0 to 148499
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reviewId              148500 non-null  object        
 1   userName              148500 non-null  object        
 2   userImage             148500 non-null  object        
 3   content               148499 non-null  object        
 4   score                 148500 non-null  int64         
 5   thumbsUpCount         148500 non-null  int64         
 6   reviewCreatedVersion  130313 non-null  object        
 7   at                    148500 non-null  datetime64[ns]
 8   replyContent          132954 non-null  object        
 9   repliedAt             132954 non-null  datetime64[ns]
 10  appVersion            130313 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 12.5+ MB


In [7]:
# Membuat DataFrame baru (clean_df) dengan menghapus baris yang memiliki nilai yang hilang (NaN) dari app_reviews_df
clean_df = app_reviews_df.dropna()

In [9]:
# Menghapus baris duplikat dari DataFrame clean_df
clean_df = clean_df.drop_duplicates()

# Menghitung jumlah baris dan kolom dalam DataFrame clean_df setelah menghapus duplikat
jumlah_ulasan_setelah_hapus_duplikat, jumlah_kolom_setelah_hapus_duplikat = clean_df.shape

Kode ini untuk menghapus baris duplikat dari DataFrame clean_df. Dengan menggunakan metode .drop_duplicates(), baris-baris yang identik akan dihapus, menjaga hanya satu baris dari setiap entri unik. Selanjutnya, kita menghitung jumlah baris dan kolom dalam DataFrame clean_df setelah proses penghapusan duplikat.

# **Preprocessing Text**

In [10]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Memecah teks menjadi daftar kata
    words = text.split()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

elanjutnya adalah penghapus kumpulan slang words atau kata-kata informal yang sering digunakan dalam percakapan sehari-hari, terutama pada media sosial atau obrolan online. Setiap kata slang memiliki padanan atau substitusi dengan kata formal atau baku. Misalnya, "abis" merupakan singkatan dari "habis", "wtb" merupakan singkatan dari "beli", dan seterusnya. Kamus ini berguna untuk membantu pemahaman dan interpretasi teks yang menggunakan bahasa informal atau slang.

In [12]:
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal"}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [15]:
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

# Membersihkan teks dan menyimpannya di kolom 'text_clean'
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
clean_df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
0,87509e9b-0076-4eb7-9c74-83ee5b8456ef,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"sangat buruk,setiap membeli kuota dengan apk i...",1,156,1.59.0,2025-01-16 14:07:40,"Hai Kak, maaf udah bikin ga nyaman ya :( Kalau...",2025-01-16 21:11:59,1.59.0,sangat buruksetiap membeli kuota dengan apk in...,sangat buruksetiap membeli kuota dengan apk in...,sangat buruksetiap membeli kuota dengan apk in...,"[sangat, buruksetiap, membeli, kuota, dengan, ...","[buruksetiap, membeli, kuota, apk, sinyal, sta...",buruksetiap membeli kuota apk sinyal stabiltid...
1,1b1ca5ac-66bd-4613-9bb4-37cb1c3318f2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Sering hilang sinyal,lemot dan parah sinyalnya...",1,10,1.60.0,2025-02-06 04:03:59,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2025-02-06 06:23:12,1.60.0,Sering hilang sinyallemot dan parah sinyalnya ...,sering hilang sinyallemot dan parah sinyalnya ...,sering hilang sinyallemot dan parah sinyalnya ...,"[sering, hilang, sinyallemot, dan, parah, siny...","[hilang, sinyallemot, parah, sinyalnya, errorn...",hilang sinyallemot parah sinyalnya errornyanye...
2,ff7da062-d90e-447c-afad-97a488dcf1ea,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Aplikasi berat banget, suka bengong. Kode prog...",1,47,1.60.0,2025-01-31 22:23:53,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2025-02-01 18:36:51,1.60.0,Aplikasi berat banget suka bengong Kode progra...,aplikasi berat banget suka bengong kode progra...,aplikasi berat banget suka bengong kode progra...,"[aplikasi, berat, banget, suka, bengong, kode,...","[aplikasi, berat, banget, suka, bengong, kode,...",aplikasi berat banget suka bengong kode progra...
3,ef265867-a243-4a97-bc6a-4209d51b2525,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Kualitas Jaringan sangat lemot dan sering terg...,1,18,1.59.0,2025-01-26 07:37:31,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2025-01-26 15:22:54,1.59.0,Kualitas Jaringan sangat lemot dan sering terg...,kualitas jaringan sangat lemot dan sering terg...,kualitas jaringan sangat lemot dan sering terg...,"[kualitas, jaringan, sangat, lemot, dan, serin...","[kualitas, jaringan, lemot, terganggu, kadang,...",kualitas jaringan lemot terganggu kadang detik...
4,02cfdf8a-a35e-449c-a92e-c5c9668dd08c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Bagi kalian yang baca ini terus jaringan byu l...,1,15,1.60.0,2025-02-02 20:34:20,"Hai Kak, maaf nih terkait kendalanya, bisa cob...",2025-02-02 23:00:18,1.60.0,Bagi kalian yang baca ini terus jaringan byu l...,bagi kalian yang baca ini terus jaringan byu l...,bagi kalian yang baca ini terus jaringan byu l...,"[bagi, kalian, yang, baca, ini, terus, jaringa...","[baca, jaringan, byu, leg, lainlain, paksa, re...",baca jaringan byu leg lainlain paksa registras...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148490,99ea26bf-9d41-47a0-bb60-3012a21b4f26,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Ok,5,0,476,2020-05-08 18:18:24,makasih buat feedback dan sarannya ya kak. Nin...,2020-05-09 04:32:28,476,Ok,ok,ok,[ok],[ok],ok
148491,beeab4d9-9017-4659-b4be-5dafce5c9f81,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Ok,5,0,476,2020-05-08 09:06:56,makasih buat feedback dan sarannya ya kak. Nin...,2020-05-09 00:56:32,476,Ok,ok,ok,[ok],[ok],ok
148494,0803695a-30d0-45df-a61b-8fd73cfbe929,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Mantuuuul,5,0,476,2020-06-05 12:05:10,Hai Kak. Makasih banyak buat feedbacknya ya. N...,2020-06-06 00:28:31,476,Mantuuuul,mantuuuul,mantuuuul,[mantuuuul],[mantuuuul],mantuuuul
148497,0ed34dea-f069-4e6c-bb64-7381f679178b,Pengguna Google,https://play-lh.googleusercontent.com/EGemo

# **Pelabelan**

In [18]:
import csv
import requests
from io import StringIO

# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

Kode di atas untuk melakukan analisis sentimen pada teks berbahasa Indonesia menggunakan kamus kata-kata positif dan negatif. Pertama, kita mengirim permintaan HTTP untuk mengambil data kamus kata-kata positif dan negatif dari GitHub.

Jika permintaan berhasil, kita membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma. Kemudian, setiap baris dalam file CSV dibaca dan kata-kata beserta skornya dimasukkan ke kamus yang sesuai (lexicon_positive untuk kata-kata positif dan lexicon_negative untuk kata-kata negatif).

In [19]:
# Fungsi untuk menentukan polaritas sentimen dari tweet

def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:

    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # Inisialisasi variabel polaritas

    if (score >= 0):
        polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 0, maka polaritas adalah positif
    elif (score < 0):
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif

    # else:
    #     polarity = 'neutral'
    # Ini adalah bagian yang bisa digunakan untuk menentukan polaritas netral jika diperlukan

    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

Setelah kamus-kamus tersebut terisi, kita mendefinisikan fungsi sentiment_analysis_lexicon_indonesia yang akan menerima teks sebagai input. Dalam fungsi ini, setiap kata pada teks akan diperiksa, ada dalam kamus positif atau negatif. Jika ada, skor sentimen akan ditambahkan atau dikurangkan sesuai dengan skor kata tersebut dalam kamus. Setelah semua kata diperiksa, skor sentimen akan digunakan untuk menentukan polaritas teks, positif atau negatif.

In [20]:
results = clean_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
positive    67712
negative    48040
Name: count, dtype: int64


# **Eksplorasi Label**

# **Data Splitting dan Ekstraksi Fitur dengan TF-IDF**

In [22]:
# Import the necessary class
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split



# Pisahkan data menjadi fitur (tweet) dan label (sentimen)
X = clean_df['text_akhir']
y = clean_df['polarity']

# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)

# Konversi hasil ekstraksi fitur menjadi dataframe
features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

# Menampilkan hasil ekstraksi fitur
features_df

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# **Modeling**

In [23]:
from sklearn.naive_bayes import BernoulliNB

# Membuat objek model Naive Bayes (Bernoulli Naive Bayes)
naive_bayes = BernoulliNB()

# Melatih model Naive Bayes pada data pelatihan
naive_bayes.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_nb = naive_bayes.predict(X_train.toarray())
y_pred_test_nb = naive_bayes.predict(X_test.toarray())

# Evaluasi akurasi model Naive Bayes
accuracy_train_nb = accuracy_score(y_pred_train_nb, y_train)
accuracy_test_nb = accuracy_score(y_pred_test_nb, y_test)

# Menampilkan akurasi
print('Naive Bayes - accuracy_train:', accuracy_train_nb)
print('Naive Bayes - accuracy_test:', accuracy_test_nb)

Naive Bayes - accuracy_train: 0.7836632433775014
Naive Bayes - accuracy_test: 0.787654960908816
